#Change runtime type to GPU

In [ ]:
# Only clone once and drag it to your google drive after next block
!git clone https://github.com/abbesmoe/Taco-Mask

In [ ]:
# Suggestion: Use google drive so everything auto saves and you dont lose progress
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# We are using older versions of keras and tensorflow to run to train
!pip install keras==2.1.6
!pip install 'h5py==2.10.0'

In [ ]:
%tensorflow_version 1.x

In [ ]:
# Change directory to /detector
# %cd /content/gdrive/MyDrive/
%cd Taco-Mask/TACO/detector

In [ ]:
# Import packages
%matplotlib inline
import csv
import dataset
import json
import matplotlib.pyplot as plt
import model as modellib
import numpy as np
import os
import pandas as pd
import skimage.io
import visualize
from config import Config
from os import path
from PIL import Image

# Train Dataset

In [ ]:
# Not necessary to run unless you change map or add more annotated images
!python3 split_dataset.py --dataset_dir ../data --nr_trials 1

In [ ]:
# Train a new model starting from pre-trained taco weights on train set split #0
!python3 -W ignore detector.py train --model=taco --dataset=../data --class_map=taco_config/map_new.csv --round 0

In [ ]:
# Continue training a model that you had trained earlier
!python3 -W ignore detector.py train  --dataset=../data --model=taco20210730T1340 --class_map=./taco_config/map_new.csv --round 0 --lrate=0.01

In [ ]:
!python3 detector.py test --dataset=../data --model=taco20210730T1340 --round 0 --class_map=./taco_config/map_new.csv

# Test new weights

In [ ]:
# check class names
class_map = {}
with open("taco_config/map_10.csv") as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]:row[1] for row in reader}

TACO_DIR = "../data/"
round = None 
subset = "train"
dataset = dataset.Taco()
taco = dataset.load_taco(TACO_DIR, round, subset, class_map=class_map, return_taco=True)

dataset.prepare()

print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("'{}',".format(info['name']))

In [ ]:
# create config and override hyperparameters
class TacoTestConfig(Config):
  BACKBONE = "resnet50"
  NAME = "taco"
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1
  NUM_CLASSES = dataset.num_classes

  # DETECTION_NMS_THRESHOLD = 0.3

config = TacoTestConfig()
config.display()

In [ ]:
# Create model and load trained weights
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_taco.hy',
                          config=config)

# models/logs/taco20210730T1340/
model.load_weights('../../mask_rcnn_taco_0100.h5',
                   by_name=True, weights_out_path=None)

In [7]:
# List class names depending on the map used
class_names = ["BG","Bottle","Bottle cap","Can","Cigarette","Cup",
               "Lid","Other","Plastic bag + wrapper","Pop tab","Straw"]
#class_names = ['BG', 'Bag & wrapper', 'Blister pack', 'Bottle cap',
#               'Can', 'Cardboard', 'Carton', 'Cigarette', 'Container',
#               'Cup', 'Food waste', 'Glass', 'Glass bottle', 'Lid',
#               'Other', 'Other plastic', 'Paper', 'Plastic bottle',
#               'Pop tab', 'Straw', 'Styrofoam', 'Tube']

In [8]:
# Select an image to test. Store in in uploaded images or grab an image that you trained on

imagePath = "../../uploaded_images/"
# imagePath = "../data/batch_0"
imageName = "1.jpg"
img = Image.open(os.path.join(imagePath, imageName))

fileName = os.path.splitext(imageName)[0]
fileExtension = os.path.splitext(imageName)[1]

if fileExtension != '.jpg':
  img.convert('RGB').save("{}{}.jpg".format(imagePath, fileName))
  imageName = "{}.jpg".format(fileName)

image = skimage.io.imread(os.path.join(imagePath, imageName))

In [ ]:
# run detection
r = model.detect([image], verbose=0)[0]

In [ ]:
# Give it a min accuracy score for it to display
def min_accuracy(r,a):
  result = {'rois': [], 'masks': [], 'class_ids': [], 'scores': []}
  indecies = []
  for i,ele in enumerate(r['scores']):
    if ele >= a:
      result['rois'].append(r['rois'][i])
      result['class_ids'].append(r['class_ids'][i])
      result['scores'].append(r['scores'][i])
      indecies.append(i)
  
  result['masks'] = r['masks'][:,:,indecies]

  result['rois'] = np.asarray(result['rois'])
  result['masks'] = np.asarray(result['masks'])
  result['class_ids'] = np.asarray(result['class_ids'])
  result['scores'] = np.asarray(result['scores'])
  return result

r = min_accuracy(r,0.9)
print(r['class_ids'])

In [ ]:
# Display original image
plt.figure(figsize=(12,10))
skimage.io.imshow(image)

In [ ]:
# Display annotated image
visualize.display_instances(image, imageName, r['rois'], r['masks'],
                            r['class_ids'], class_names, r['scores'])